In [1]:
from pathlib import Path

from agno.agent import Agent
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase

* Models

In [2]:
from agno.models.ollama import Ollama

ollama_model=Ollama(id="llama3.2:3b")

In [3]:
from agno.embedder.ollama import OllamaEmbedder

ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)


* Vector Database

In [5]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [6]:
# Create CSV knowledge base
# Create a knowledge base of PDFs from URLs
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load()

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Added 14 documents to knowledge base

In [7]:
knowledge_base

PDFUrlKnowledgeBase(reader=PDFUrlReader(chunk=True, chunk_size=3000, separators=['\n', '\n\n', '\r', '\r\n', '\n\r', '\t', ' ', '  '], chunking_strategy=<agno.document.chunking.fixed.FixedSizeChunking object at 0x0000019C9716AE90>), vector_db=<agno.vectordb.lancedb.lance_db.LanceDb object at 0x0000019CFFE84EC0>, num_documents=5, optimize_on=1000, chunking_strategy=<agno.document.chunking.fixed.FixedSizeChunking object at 0x0000019C9716AE90>, urls=['https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf'])

* Agent with knowledge

In [9]:
# Create and use the agent
agent = Agent(model=ollama_model,
            knowledge=knowledge_base,  
              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
            search_knowledge=True,
            show_tool_calls=True,
            debug_mode=True,
            markdown=False)


* Tests

In [10]:
agent.run("How do I make chicken and galangal in coconut milk soup")

DEBUG    *********** Agent ID: eb3265c7-2c9c-4df4-b8bf-da5acd5c814b ***********

DEBUG    *********** Session ID: 6f042532-415f-465b-8609-8123869b5bf0 ***********

DEBUG    *********** Agent Run Start: e239a6d6-3319-4f3b-94d1-13115016f981 ***********

DEBUG    Processing tools for model

DEBUG    Included function search_knowledge_base

...

DEBUG    Added 4 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: e239a6d6-3319-4f3b-94d1-13115016f981 ***********

RunResponse(content=" - Running: search_knowledge_base(query=chicken and galangal in coconut milk soup recipe)\n\nHere are the recipes extracted from the provided Thai cuisine book:\n\n**Recipe 1: Khao Niew Dam Piek Maphrao (Awn Black Glutinous Rice Pudding with Young Coconut Flesh)**\n\nIngredients:\n\n* 1 cup black glutinous rice\n* 2 cups water\n* 1/4 cup coconut milk\n* 1/4 cup sugar\n* 1/4 teaspoon salt\n* 1/2 cup young coconut flesh\n\nInstructions:\n\n1. Rinse the black glutinous rice and soak it in water for at least 30 minutes.\n2. Drain and rinse the rice again.\n3. In a medium saucepan, combine the rice, water, sugar, and salt. Cook over medium heat, stirring constantly, until the mixture comes to a boil.\n4. Reduce the heat to low and simmer for about 10-15 minutes or until the rice is cooked and creamy.\n5. Stir in the coconut milk and cook for another minute.\n6. Transfer the pudding to individual serving bowls and top with salted coconut cream.\n\n**Recipe 2: Tom Yum Goo